### Importing libraries

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

### Loading Dataset (sonnets of shakespear)

In [2]:
text=(open("./sonnets.txt").read())
text=text.lower()

### Chart to vector map

In [3]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

### Preparing Data

In [4]:
x = []
y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    x.append([char_to_n[char] for char in sequence])
    y.append(char_to_n[label])

### Data reformation

In [5]:
x_modified = np.reshape(x, (len(x), seq_length, 1))
x_modified = x_modified / float(len(characters))
y_modified = np_utils.to_categorical(y)

In [6]:
y_modified.shape

(119649, 57)

### Model

In [7]:
model = Sequential()
model.add(LSTM(700, input_shape=(x_modified.shape[1], x_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 700)          1965600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 700)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 700)          3922800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 700)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 700)               3922800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 57)                39957     
Total para

In [ ]:
model.fit(x_modified, y_modified, epochs=100, batch_size=50)

model.save('./poemModel.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
   500/119649 [..............................] - ETA: 3:16:47 - loss: 3.8162

### Generating Text

In [ ]:
string_mapped = x[99]
full_string = [n_to_char[value] for value in string_mapped]

for i in range(400):
    xtest = np.reshape(string_mapped,(1,len(string_mapped), 1))
    xtest = xtest / float(len(characters))

    pred_index = np.argmax(model.predict(xtest, verbose=0))
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

### Output

In [ ]:
txt=""
for char in full_string:
    txt = txt+char
txt